### Name: Swathi Subramanyam Pabbathi

### Date: 02/07/2026


### Assignment 5.1: ML System Observability - Exercise

### Environment Setup and Imports

In [2]:
import os, sys, json, time
import boto3
import pandas as pd
import numpy as np
from dotenv import load_dotenv

import sagemaker
from sagemaker.session import Session
from sagemaker.model import ModelPackage
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.model_monitor import EndpointInput

from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import StringDeserializer
from sagemaker.predictor import Predictor

sys.path.append("..")
from config.config import BUCKET_NAME, S3_PREFIX, AWS_REGION, TARGET_COLUMN, MODEL_NAME

print("SageMaker SDK:", sagemaker.__version__)
print("Region:", AWS_REGION)

SageMaker SDK: 2.256.1
Region: us-east-1


In [3]:
from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

In [4]:
load_dotenv()
ROLE_ARN = os.getenv("SAGEMAKER_ROLE_ARN")
if not ROLE_ARN:
    raise RuntimeError("SAGEMAKER_ROLE_ARN not found in .env")

In [5]:
# Sessions / clients
boto_sess = boto3.Session(region_name=AWS_REGION)
sm_sess = Session(boto_session=boto_sess)

sm_client = boto3.client("sagemaker", region_name=AWS_REGION)
s3_client = boto3.client("s3", region_name=AWS_REGION)

bucket = BUCKET_NAME

In [6]:

# Load latest training run metadata (so no hardcoding)

prefix = "nfci-xgboost-regression"  # same as training notebook prefix
metadata_key = f"{prefix}/run_metadata/latest.json"

obj = s3_client.get_object(Bucket=bucket, Key=metadata_key)
run_metadata = json.loads(obj["Body"].read())

job_name = run_metadata["training_job_name"]
training_prefix = run_metadata["training_prefix"]

print("Loaded job_name:", job_name)
print("Loaded prefix:", training_prefix)

Loaded job_name: xgb-nfci-reg-2026-02-08-12-30-10
Loaded prefix: nfci-xgboost-regression


### Retrieve Latest Model from SageMaker Model Registry

In [7]:
# Get latest approved/pending model package from Model Registry
model_package_group = f"{MODEL_NAME}-group"   # IMPORTANT: must match your evaluation notebook
resp = sm_client.list_model_packages(
    ModelPackageGroupName=model_package_group,
    SortBy="CreationTime",
    SortOrder="Descending",
    MaxResults=1
)

pkgs = resp["ModelPackageSummaryList"]
if not pkgs:
    raise ValueError(f"No model packages found in group: {model_package_group}")

latest_model = pkgs[0]
model_package_arn = latest_model["ModelPackageArn"]

print("Latest model package ARN:", model_package_arn)
print("Version:", latest_model["ModelPackageVersion"])
print("Approval:", latest_model["ModelApprovalStatus"])

Latest model package ARN: arn:aws:sagemaker:us-east-1:222634372778:model-package/nfci-xgboost-group/2
Version: 2
Approval: Approved


### Deploy Endpoint with Data Capture Enabled

In [13]:

# Deploy endpoint with Data Capture enabled

endpoint_name = f"nfci-xgb-endpoint-{job_name}"[:63]  # endpoint name max length 63

data_capture_prefix = f"{training_prefix}/data-capture/{endpoint_name}"
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f"s3://{bucket}/{data_capture_prefix}",
    capture_options=["Input", "Output"],
    csv_content_types=["text/csv"],
)

model = ModelPackage(
    role=ROLE_ARN,
    model_package_arn=model_package_arn,
    sagemaker_session=sm_sess,
)

print("Deploying endpoint:", endpoint_name)
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)


print("Endpoint deployed:", endpoint_name)
print("Data capture:", f"s3://{bucket}/{data_capture_prefix}")

Deploying endpoint: nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10
------!Endpoint deployed: nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10
Data capture: s3://nfci-forecasting-222634372778/nfci-xgboost-regression/data-capture/nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10


In [14]:
import time
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import StringDeserializer

# wait for endpoint
while True:
    desc = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = desc["EndpointStatus"]
    print("EndpointStatus:", status)
    if status == "InService":
        break
    if status == "Failed":
        raise RuntimeError("Endpoint deployment failed: " + str(desc))
    time.sleep(20)

# Create predictor manually
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sm_sess
)

predictor.serializer = CSVSerializer()
predictor.deserializer = StringDeserializer()

print("Predictor ready:", predictor.endpoint_name)

EndpointStatus: InService
Predictor ready: nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10


### Load Production Dataset

In [15]:
# 6) Send a few inference requests (to generate capture logs)
test_s3 = f"s3://{bucket}/{S3_PREFIX['test']}/features.parquet"
df_test = pd.read_parquet(test_s3)

In [16]:
import io
obj = s3_client.get_object(
    Bucket=BUCKET_NAME,
    Key='data/splits/production/features.parquet'
)
production_df = pd.read_parquet(io.BytesIO(obj['Body'].read()))

print(f"Production data shape: {production_df.shape}")
print(f"Date range: {production_df['date'].min()} to {production_df['date'].max()}")
print(f"States: {production_df['state_fips'].nunique()}")
print(f"\nColumns: {list(production_df.columns)[:10]}...")

Production data shape: (2400, 80)
Date range: 2020-01-01 00:00:00 to 2023-12-01 00:00:00
States: 50

Columns: ['state_fips', 'date', 'UNRATE', 'PAYEMS', 'CIVPART', 'EMRATIO', 'U6RATE', 'AWHMAN', 'AHETPI', 'CPIAUCSL']...


### Generate Sample Predictions (Trigger Data Capture)

In [17]:
# Get actual NFCI values from production period
# These are the ground truth values to compare against prediction
actual_nfci = production_df.groupby('date')['NFCI'].first().sort_index()

print(f"Production period NFCI values: {len(actual_nfci)} months")
print(f"\nFirst 5 values:")
print(actual_nfci.head())

Production period NFCI values: 48 months

First 5 values:
date
2020-01-01   -0.61879
2020-02-01   -0.29288
2020-03-01    0.28082
2020-04-01    0.16847
2020-05-01   -0.26647
Name: NFCI, dtype: float64


In [18]:
# Drop non-numeric columns like training
df_test = df_test.drop(columns=["date", "Datetime"], errors="ignore")

In [19]:
# Separate y and X
y_true = df_test[TARGET_COLUMN].values
X = df_test.drop(columns=[TARGET_COLUMN])

In [21]:
# Send first 5 rows
for i in range(5):
    row = X.iloc[i].tolist()
    resp = predictor.predict(row)  # CSV row
    print("pred raw:", resp)

print("Waiting 60s for capture logs to land in S3...")
time.sleep(60)

pred raw: {"predictions": [{"score": -0.5691239833831787}]}
pred raw: {"predictions": [{"score": -0.4638028144836426}]}
pred raw: {"predictions": [{"score": -0.5707794427871704}]}
pred raw: {"predictions": [{"score": -0.572127103805542}]}
pred raw: {"predictions": [{"score": -0.5712562203407288}]}
Waiting 60s for capture logs to land in S3...


### Create Baseline Dataset for Model Monitor

In [22]:

# Create baseline for Model Monitor (Data Quality / Drift)

# Baseline should match captured INPUT format.
# Used the same CSV you upload for training inference format (features only).
# We'll create a baseline CSV from training parquet (features only).

train_parquet_s3 = f"s3://{bucket}/{S3_PREFIX['train']}/features.parquet"
df_train = pd.read_parquet(train_parquet_s3)
df_train = df_train.drop(columns=["date", "Datetime"], errors="ignore")

X_train = df_train.drop(columns=[TARGET_COLUMN], errors="ignore")

baseline_local = "baseline.csv"
X_train.to_csv(baseline_local, index=False, header=False)

baseline_s3_uri = sm_sess.upload_data(baseline_local, key_prefix=f"{training_prefix}/monitoring/baseline")
print("Baseline dataset uploaded:", baseline_s3_uri)

monitor = DefaultModelMonitor(
    role=ROLE_ARN,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=sm_sess,
)

baseline_results_uri = f"s3://{bucket}/{training_prefix}/monitoring/baseline-results/{job_name}"

monitor.suggest_baseline(
    baseline_dataset=baseline_s3_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=baseline_results_uri,
    wait=True,
)

print("Baseline generated at:", baseline_results_uri)
print("Constraints:", monitor.baseline_constraints())
print("Statistics:", monitor.baseline_statistics())

INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2026-02-08-18-26-45-726


Baseline dataset uploaded: s3://sagemaker-us-east-1-222634372778/nfci-xgboost-regression/monitoring/baseline/baseline.csv
.......................2026-02-08 18:27:54.790463: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-02-08 18:27:54.790505: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2026-02-08 18:27:56.500836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2026-02-08 18:27:56.500894: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2026-02-08 18:27:56.500922: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:38                                                                                   │
│                                                                                                  │
│   35 )                                                                                           │
│   36                                                                                             │
│   37 print("Baseline generated at:", baseline_results_uri)                                       │
│ ❱ 38 print("Constraints:", monitor.baseline_constraints())                                       │
│   39 print("Statistics:", monitor.baseline_statistics())                                         │
│   40                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'DefaultModelMonitor' object has no attribute 'baseline_constraints'

In [23]:
constraints_s3_uri = f"{baseline_results_uri}/constraints.json"
statistics_s3_uri  = f"{baseline_results_uri}/statistics.json"

print("constraints:", constraints_s3_uri)
print("statistics :", statistics_s3_uri)

constraints: s3://nfci-forecasting-222634372778/nfci-xgboost-regression/monitoring/baseline-results/xgb-nfci-reg-2026-02-08-12-30-10/constraints.json
statistics : s3://nfci-forecasting-222634372778/nfci-xgboost-regression/monitoring/baseline-results/xgb-nfci-reg-2026-02-08-12-30-10/statistics.json


### Compare Actual vs Predicted Values (Sanity Check)

In [24]:
# compare predictions versus actuals

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1) clean production df like training
prod = production_df.drop(columns=["date", "Datetime"], errors="ignore").copy()

# 2) y_true (ground truth)
y_true_prod = prod[TARGET_COLUMN].values

# 3) X features only
X_prod = prod.drop(columns=[TARGET_COLUMN], errors="ignore")

print("X_prod shape:", X_prod.shape, "y_true shape:", y_true_prod.shape)

X_prod shape: (2400, 78) y_true shape: (2400,)


### predicted Vs Actuals

In [25]:
import json
import numpy as np
import pandas as pd

prod_small = production_df.drop(columns=["date", "Datetime"], errors="ignore").copy()

y_actual = prod_small[TARGET_COLUMN].values[:5]
X_small = prod_small.drop(columns=[TARGET_COLUMN]).iloc[:5]

preds = []
for i in range(5):
    row = X_small.iloc[i].tolist()
    pred_text = predictor.predict(row)  # string JSON
    
    pred_obj = json.loads(pred_text)
    score = pred_obj["predictions"][0]["score"]
    preds.append(float(score))

y_pred = np.array(preds)

pd.DataFrame({"Actual": y_actual, "Predicted": y_pred})

,Actual,Predicted
0,-0.61879,-0.561555
1,-0.29288,-0.576910
2,0.28082,-0.213693
3,0.16847,-0.102921
4,-0.26647,-0.126143


#### Verify Captured Inference Logs in S3

In [26]:
# Check captured data
response = s3_client.list_objects_v2(
    Bucket=BUCKET_NAME,
    Prefix=data_capture_prefix,
    MaxKeys=10
)

print(f"\nCaptured data files:")
for obj in response.get('Contents', []):
    print(f"  {obj['Key']} ({obj['Size']} bytes)")


Captured data files:
  nfci-xgboost-regression/data-capture/nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10/nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10/AllTraffic/2026/02/08/12/52-13-957-834adfa3-f283-4eec-9836-2ace7f5de07f.jsonl (5857 bytes)
  nfci-xgboost-regression/data-capture/nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10/nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10/AllTraffic/2026/02/08/13/12-03-855-13ef6c0f-a8bf-4d88-8b43-64e406e50335.jsonl (6876 bytes)
  nfci-xgboost-regression/data-capture/nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10/nfci-xgb-endpoint-xgb-nfci-reg-2026-02-08-12-30-10/AllTraffic/2026/02/08/18/23-00-494-0a02aad4-3c35-43ca-882e-9dd58a57b254.jsonl (5857 bytes)


### Define Custom CloudWatch Metric Publisher

In [27]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

# Create Model Monitor instance
monitor = DefaultModelMonitor(
    role=ROLE_ARN,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

print("Model Monitor instance created")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


Model Monitor instance created


### Compute Production Metrics (RMSE & MAE)

In [28]:
# Check built-in SageMaker endpoint metrics
print("Built-in SageMaker Endpoint Metrics:")

metrics_to_check = [
    'Invocations',
    'InvocationsPerInstance', 
    'ModelLatency',
    'OverheadLatency',
    'Invocation4XXErrors',
    'Invocation5XXErrors',
]

for metric_name in metrics_to_check:
    print(f"  AWS/SageMaker: {metric_name}")

Built-in SageMaker Endpoint Metrics:
  AWS/SageMaker: Invocations
  AWS/SageMaker: InvocationsPerInstance
  AWS/SageMaker: ModelLatency
  AWS/SageMaker: OverheadLatency
  AWS/SageMaker: Invocation4XXErrors
  AWS/SageMaker: Invocation5XXErrors


### Publish Row-Level Metrics to CloudWatch


In [29]:
# Publish custom metrics for NFCI forecasting
def publish_forecast_metrics(forecast_mean, actual_value=None):
    """
    Publish custom CloudWatch metrics for NFCI forecasting.
    """
    timestamp = datetime.utcnow()
    
    metrics = [
        {
            'MetricName': 'ForecastMean',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'Model', 'Value': 'DeepAR'},
            ],
            'Timestamp': timestamp,
            'Value': float(forecast_mean[0]),  # First month forecast
            'Unit': 'None'
        },
    ]
    
    # If actuals exists, compute and publish error
    if actual_value is not None:
        error = abs(forecast_mean[0] - actual_value)
        metrics.append({
            'MetricName': 'ForecastError',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'Model', 'Value': 'DeepAR'},
            ],
            'Timestamp': timestamp,
            'Value': error,
            'Unit': 'None'
        })
    
    cw_client.put_metric_data(
        Namespace='NFCI-Forecasting',
        MetricData=metrics
    )
    
    return metrics

print("Custom metric function defined")

Custom metric function defined


In [30]:
sm_client = boto3.client("sagemaker", region_name=AWS_REGION)
s3_client = boto3.client("s3", region_name=AWS_REGION)
cw_client = boto3.client("cloudwatch", region_name=AWS_REGION)

### Publish Summary Production Metrics to CloudWatch

In [31]:
from datetime import datetime

# Send predictions and publish metrics with actual values
print("Publishing production metrics to CloudWatch...\n")

# Use the predictions and actuals we already computed
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
mae = mean_absolute_error(y_actual, y_pred)

print("RMSE:", rmse)
print("MAE :", mae)

print("\nPublishing production metrics to CloudWatch...\n")

for i in range(len(y_actual)):
    metrics = publish_forecast_metrics(
        forecast_mean=[y_pred[i]],
        actual_value=y_actual[i]
    )

    error = abs(y_pred[i] - y_actual[i])
    print(f"Row {i+1}: Predicted={y_pred[i]:.4f}, Actual={y_actual[i]:.4f}, Error={error:.4f}")
    time.sleep(1)

# Also publish summary metrics
cw_client.put_metric_data(
    Namespace='NFCI-Forecasting',
    MetricData=[
        {
            'MetricName': 'ProductionRMSE',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'Model', 'Value': 'DeepAR'},
            ],
            'Value': rmse,
            'Unit': 'None'
        },
        {
            'MetricName': 'ProductionMAE',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'Model', 'Value': 'DeepAR'},
            ],
            'Value': mae,
            'Unit': 'None'
        },
    ]
)

print(f"\nPublished ProductionRMSE={rmse:.4f}, ProductionMAE={mae:.4f}")

Publishing production metrics to CloudWatch...

RMSE: 0.2904604507216057
MAE : 0.2494990851368904

Publishing production metrics to CloudWatch...

Row 1: Predicted=-0.5616, Actual=-0.6188, Error=0.0572
Row 2: Predicted=-0.5769, Actual=-0.2929, Error=0.2840
Row 3: Predicted=-0.2137, Actual=0.2808, Error=0.4945
Row 4: Predicted=-0.1029, Actual=0.1685, Error=0.2714
Row 5: Predicted=-0.1261, Actual=-0.2665, Error=0.1403

Published ProductionRMSE=0.2905, ProductionMAE=0.2495
